In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn import  preprocessing, ensemble
from sklearn.metrics import log_loss,accuracy_score
from sklearn.cross_validation import KFold
import re
import string
from collections import defaultdict, Counter


In [33]:
#try xgboost
#fucntion from SRK
def runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, \
     seed_val=0, early_stop = 20,num_rounds=10000, eta = 0.1, max_depth = 6):
    param = {}
    param['objective'] = 'multi:softprob'
    param['eta'] = eta
    param['max_depth'] = max_depth
    param['silent'] = 1
    param['num_class'] = 3
    param['eval_metric'] = "mlogloss"
    param['min_child_weight'] = 1
    param['subsample'] = 0.7
    param['colsample_bytree'] = 0.7
    param['seed'] = seed_val
    num_rounds = num_rounds

    plst = list(param.items())
    xgtrain = xgb.DMatrix(train_X, label=train_y,feature_names=feature_names)

    if test_y is not None:
        xgtest = xgb.DMatrix(test_X, label=test_y,feature_names=feature_names)
        watchlist = [ (xgtrain,'train'), (xgtest, 'test') ]
        model = xgb.train(plst, xgtrain, num_rounds, watchlist,\
        early_stopping_rounds=early_stop)
    else:
        xgtest = xgb.DMatrix(test_X,feature_names=feature_names)
        model = xgb.train(plst, xgtrain, num_rounds)

    pred_test_y = model.predict(xgtest)
    return pred_test_y, model


In [3]:
#feature processing functions
#define punctutaion filter
def removePunctuation(x):
    #filter the head or tail blanks
    x = re.sub(r'^\s+',r' ',x)
    x = re.sub(r'\s+$',r' ',x)
    
    # Lowercasing all words
    x = x.lower()
    # Removing non ASCII chars, warning if you are dealing with other languages!!!!!!!!!!!!!!!
    x = re.sub(r'[^\x00-\x7f]',r' ',x)
    #change all the blank to space
    x = re.sub(r'\s',r' ',x)
    # Removing (replacing with empty spaces actually) all the punctuations
    removing = string.punctuation#.replace('-','')# except '-'
    removed = re.sub("["+removing+"]", "", x)
    #removing the line-changing
    #removed = re.sub('\\n'," ",removed)    
    return removed

#feature processing functions
def proecessStreet(address):
    #remove the building number
    pattern = re.compile('^[\d-]*[\s]+')
    street = removePunctuation(pattern.sub('',address))
    
    #sub the st to street
    pattern = re.compile('( st)$')
    street = pattern.sub(' street',street)
    
    #sub the ave to avenue
    pattern = re.compile('( ave)$')
    street = pattern.sub(' avenue',street)
    
    pattern = re.compile('(\d+)((th)|(st)|(rd)|(nd))')
    street = pattern.sub('\g<1>',street)
    
    #deal with the w 14 street => west 14 street
    pattern = re.compile('(w)(\s+)(\d+)')    
    street = pattern.sub('west \g<3>',street)
    
    #deal with the e....
    pattern = re.compile('(e)(\s+)(\d+)')    
    street = pattern.sub('east \g<3>',street)
    
    return street
    
#from "this is a lit"s python version by rakhlin
def singleValueConvert(df1,df2,column,minimum_size=5):
    ps = df1[column].append(df2[column])
    grouped = ps.groupby(ps).size().to_frame().rename(columns={0: "size"})
    df1.loc[df1.join(grouped, on=column, how="left")["size"] <= minimum_size, column] = -1
    df2.loc[df2.join(grouped, on=column, how="left")["size"] <= minimum_size, column] = -1
    return df1, df2

def manager_skill_eval(train_df,test_df,unrank_threshold = 10):

    target_num_map = {'High':2, 'Medium':1, 'Low':0}
    temp=pd.concat([train_df.manager_id,pd.get_dummies(train_df.interest_level)], axis = 1).groupby('manager_id').mean()
     
    temp.columns = ['ManHigh','ManLow', 'ManMedium']
    
    print temp.columns
    temp['count'] = train_df.groupby('manager_id').count().iloc[:,1]
    
    temp['manager_skill'] = temp['ManHigh']*2 + temp['ManMedium']
    
    #ixes of the managers with to few sample
    unranked_managers_ixes = temp['count']<unrank_threshold
    ranked_managers_ixes = ~unranked_managers_ixes
    
    #test for using rank or unrank part for the filling values
    mean_values = temp.loc[unranked_managers_ixes, ['ManHigh','ManLow', 'ManMedium','manager_skill']].mean()
    mean_values_total = temp.loc[:, ['ManHigh','ManLow', 'ManMedium','manager_skill']].mean()
    
    #reset their values to their average
    temp.loc[unranked_managers_ixes,['ManHigh','ManLow', 'ManMedium','manager_skill']] = mean_values.values
    
    #assign the features for the train set
    new_train_df = train_df.merge(temp.reset_index(),how='left', left_on='manager_id', right_on='manager_id')
    
    #assign the features for the test/val set
    new_test_df = test_df.merge(temp.reset_index(),how='left', left_on='manager_id', right_on='manager_id')
    new_manager_ixes = new_test_df['ManHigh'].isnull()
    new_test_df.loc[new_manager_ixes,['ManHigh','ManLow', 'ManMedium','manager_skill']] = mean_values_total.values           
    
    return new_train_df,new_test_df


In [4]:
#functions for features
def featureList(train_df,test_df,limit = 0.001):
    #acquiring the feature lists
    features_in_train = train_df["features"].apply(pd.Series).unstack().reset_index(drop = True).dropna().value_counts()
    features_in_test = test_df["features"].apply(pd.Series).unstack().reset_index(drop = True).dropna().value_counts()
    
    filtered_features_in_train = features_in_train[features_in_train > limit*len(train_df)]
    filtered_features_in_test = features_in_test[features_in_test > limit*len(test_df)]
    accept_list = set(filtered_features_in_train.index).union(set(filtered_features_in_test.index))
    return accept_list

def featureMapping(train_df,test_df,feature_list):
    for feature in feature_list:
        #add the feature column for both
        #if feature in the row, then set the value for (row,feature) to 1
        train_df['with_'+feature]=train_df['features'].apply(lambda x : 1 if feature in x else 0)
        test_df['with_'+feature]=test_df['features'].apply(lambda x : 1 if feature in x else 0)
    return


In [5]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [6]:
#lodaing data
data_path = "../../kaggleData/2sigma/"
train_file = data_path + "train.json"
test_file = data_path + "test.json"
train_df = pd.read_json(train_file)
test_df = pd.read_json(test_file)
print(train_df.shape)
print(test_df.shape)


(49352, 15)
(74659, 14)


In [7]:
#basic numerical features
features_to_use  = ["bathrooms", "bedrooms", "latitude", "longitude", "price"]

In [8]:
#some new numerical features related to the price
train_df["price_per_bath"] =  (train_df["price"]*1.0/train_df["bathrooms"]).replace(np.Inf,-1)
train_df["price_per_bed"] = (train_df["price"]*1.0/train_df["bedrooms"]).replace(np.Inf,-1)
train_df["bath_per_bed"] = (train_df["bathrooms"]*1.0/train_df["bedrooms"]).replace(np.Inf,-1)
train_df["price_per_room"] = (train_df["price"]*1.0/(train_df["bedrooms"]+train_df["bathrooms"])).replace(np.Inf,-1)

test_df["price_per_bath"] =  (test_df["price"]*1.0/test_df["bathrooms"]).replace(np.Inf,-1)
test_df["price_per_bed"] = (test_df["price"]*1.0/test_df["bedrooms"]).replace(np.Inf,-1)
test_df["bath_per_bed"] = (test_df["bathrooms"]*1.0/test_df["bedrooms"]).replace(np.Inf,-1)
test_df["price_per_room"] = (test_df["price"]*1.0/(test_df["bedrooms"]+test_df["bathrooms"])).replace(np.Inf,-1)

features_to_use.extend(["price_per_bed","bath_per_bed","price_per_room"])
#features_to_use.append('price_per_bed')

In [9]:
#some transfromed features
# count of photos #
train_df["num_photos"] = train_df["photos"].apply(len)
test_df["num_photos"] = test_df["photos"].apply(len)

# count of "features" #
train_df["num_features"] = train_df["features"].apply(len)
test_df["num_features"] = test_df["features"].apply(len)

# count of words present in description column #
train_df["num_description_words"] = train_df["description"].apply(lambda x: len(x.split(" ")))
test_df["num_description_words"] = test_df["description"].apply(lambda x: len(x.split(" ")))

# convert the created column to datetime object so as to extract more features 
train_df["created"] = pd.to_datetime(train_df["created"])
test_df["created"] = pd.to_datetime(test_df["created"])

# Let us extract some features like year, month, day, hour from date columns #
train_df["created_year"] = train_df["created"].dt.year
test_df["created_year"] = test_df["created"].dt.year
train_df["created_month"] = train_df["created"].dt.month
test_df["created_month"] = test_df["created"].dt.month
train_df["created_day"] = train_df["created"].dt.day
test_df["created_day"] = test_df["created"].dt.day
train_df["created_hour"] = train_df["created"].dt.hour
test_df["created_hour"] = test_df["created"].dt.hour

# adding all these new features to use list # "listing_id",
features_to_use.extend(["num_photos", "num_features", "num_description_words","created_year","listing_id", "created_month", "created_day", "created_hour"])

In [10]:
"""
new categorical data append and converting label dummies for future use
"""
#new feature for the street_address, use them instead of the original one
train_df["street_name"] = train_df["street_address"].apply(proecessStreet)
test_df["street_name"] = test_df["street_address"].apply(proecessStreet)


In [11]:
#dealing with features

#preprocessing for features
train_df["features"] = train_df["features"].apply(lambda x:["_".join(i.split(" ")).lower().strip().replace('-','_') \
                                                            for i in x])
test_df["features"] = test_df["features"].apply(lambda x:["_".join(i.split(" ")).lower().strip().replace('-','_')\
                                                          for i in x])
#create the accept list
accept_list = list(featureList(train_df,test_df,limit = 0.001))

#map the feature to dummy slots
featureMapping(train_df,test_df,accept_list)
features_to_use.extend(map(lambda x : 'with_'+x,accept_list))

In [18]:
#use this one to generate the similarity between display and street address
train_df['detail_simi'] = map(similar,train_df["display_address"],train_df["street_address"])
train_df['inSameStreet'] = train_df['detail_simi'].apply(lambda x : 1 if x > 0.5 else 0)
test_df['detail_simi'] = map(similar,test_df["display_address"],test_df["street_address"])
test_df['inSameStreet'] = test_df['detail_simi'].apply(lambda x : 1 if x > 0.5 else 0)

features_to_use.append('inSameStreet')

In [12]:
#prepare for training
target_num_map = {'high':0, 'medium':1, 'low':2}

train_y = np.array(train_df['interest_level'].apply(lambda x: target_num_map[x]))

KF=KFold(len(train_df),5,shuffle=True,random_state = 42)

train_df = train_df.fillna(-1)
test_df = test_df.fillna(-1)

In [13]:
features_to_use.append('manager_skill')
categorical = ["display_address", "manager_id", "building_id", "street_address","street_name"]
features_to_use.extend(categorical)

In [37]:


cv_scores = []

mini_ranking = 15

eta = 0.1
md = 4
early = 64

for dev_index, val_index in KF:
        #split the orginal train set into dev_set and val_set
        dev_set, val_set = train_df.iloc[dev_index,:] , train_df.iloc[val_index,:] 
        
        #special feature engineering for the trainset
        
        
#====================================================================        
        """feature engineerings for the categorical features"""
        
        dev_set, val_set =manager_skill_eval(dev_set,val_set,\
        unrank_threshold = mini_ranking)
        
        
        #fill substitute the small size values by their mean
        for f in categorical:
            dev_set,val_set  = singleValueConvert(dev_set,val_set,f,mini_ranking)
        
            if dev_set[f].dtype=='object':
                #print(f)
                lbl = preprocessing.LabelEncoder()
                lbl.fit(list(dev_set[f])+list(val_set[f]))
                dev_set[f] = lbl.transform(list(dev_set[f].values))
                val_set[f] = lbl.transform(list(val_set[f].values))
        
#===================================================================
                
        #filter the features
        dev_X, val_X = dev_set[features_to_use].as_matrix(), val_set[features_to_use].as_matrix()
        dev_y, val_y = train_y[dev_index], train_y[val_index]
        
        """
        runXGB(train_X, train_y, test_X, test_y=None, feature_names=None, \
        seed_val=0, early_stop = 20,num_rounds=10000, eta = 0.1, max_depth = 6)
        """        
        
        preds, model = runXGB(dev_X, dev_y, val_X, val_y,\
        feature_names=features_to_use,early_stop=100,
        num_rounds = 20000, eta = eta,max_depth = 7)
        
        #using rf for feature choosing
        #model = ensemble.RandomForestClassifier(500,random_state = 42,max_depth = 30\
        #                                        ,class_weight='balanced')
        #model.fit(dev_X,dev_y)
        #preds = model.predict_proba(val_X)
        #pred = model.predict(val_X)
            
        cv_scores.append(log_loss(val_y, preds))
        break
print cv_scores
#print accuracy_score(val_y,pred)

Index([u'ManHigh', u'ManLow', u'ManMedium'], dtype='object')
[0]	train-mlogloss:1.03334	test-mlogloss:1.03657
Multiple eval metrics have been passed: 'test-mlogloss' will be used for early stopping.

Will train until test-mlogloss hasn't improved in 100 rounds.
[1]	train-mlogloss:0.976014	test-mlogloss:0.981768
[2]	train-mlogloss:0.926441	test-mlogloss:0.934753
[3]	train-mlogloss:0.884728	test-mlogloss:0.895334
[4]	train-mlogloss:0.847608	test-mlogloss:0.860455
[5]	train-mlogloss:0.815293	test-mlogloss:0.830604
[6]	train-mlogloss:0.787764	test-mlogloss:0.804605
[7]	train-mlogloss:0.763158	test-mlogloss:0.78179
[8]	train-mlogloss:0.742325	test-mlogloss:0.761995
[9]	train-mlogloss:0.722178	test-mlogloss:0.743503
[10]	train-mlogloss:0.70373	test-mlogloss:0.726747
[11]	train-mlogloss:0.686998	test-mlogloss:0.711836
[12]	train-mlogloss:0.671849	test-mlogloss:0.698665
[13]	train-mlogloss:0.658748	test-mlogloss:0.68697
[14]	train-mlogloss:0.646415	test-mlogloss:0.676319
[15]	train-mlogloss:0.

In [16]:
#features_to_use.append('manager_skill')
#categorical = ["display_address", "manager_id", "building_id", "street_address","street_name"]
#features_to_use.extend(categorical)
#features_to_use.extend(['diff_price','diff_price_per_bed','diff_price_per_bath','diff_price_per_room'])

#====================================================================        
"""feature engineerings for the categorical features"""

train_set, test_set =manager_skill_eval(train_df,test_df,\
unrank_threshold = mini_ranking)


#fill substitute the small size values by their mean
for f in categorical:
    train_set,test_set  = singleValueConvert(train_set,test_set,f,mini_ranking)

    if train_set[f].dtype=='object':
        #print(f)
        lbl = preprocessing.LabelEncoder()
        lbl.fit(list(train_df[f])+list(test_df[f]))
        train_set[f] = lbl.transform(list(train_set[f].values))
        test_set[f] = lbl.transform(list(test_set[f].values))

addAvgDiff(train_set,test_set,nn=15)

#===================================================================

train_X = train_set[features_to_use]
test_X = test_set[features_to_use]

train_X_m = train_X.as_matrix()
test_X_m = test_X.as_matrix()

preds, model = runXGB(train_X_m, train_y, test_X_m, num_rounds=243)
out_df = pd.DataFrame(preds)
out_df.columns = ["high", "medium", "low"]
out_df["listing_id"] = test_df.listing_id.values
out_df.to_csv("xgb_beta1point251-nndiff.csv", index=False)

Index([u'ManHigh', u'ManLow', u'ManMedium'], dtype='object')


C:\Users\dell\Anaconda2\lib\site-packages\ipykernel\__main__.py:123: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [22]:
dev_set.columns

Index([      u'bathrooms',        u'bedrooms',     u'building_id',
               u'created',     u'description', u'display_address',
              u'features',  u'interest_level',        u'latitude',
            u'listing_id',
       ...
            u'with_patio',     u'with_no_fee',  u'with_bike_room',
           u'detail_simi',    u'inSameStreet',         u'ManHigh',
                u'ManLow',       u'ManMedium',           u'count',
         u'manager_skill'],
      dtype='object', length=123)

In [30]:
#ananlysis by the feature importance by weight
weight = model.get_score()
weight_list = []
total = sum(weight.values())
for key in weight:
    weight[key] = weight[key]*1.0/total
    weight_list.append((key,weight[key]))
sorted(weight_list,key = lambda x:x[1],reverse = True)

[('listing_id', 0.08439382103167603),
 ('price', 0.07902628329511378),
 ('latitude', 0.07849708943376257),
 ('longitude', 0.0742635385429529),
 ('price_per_room', 0.07249955900511554),
 ('num_description_words', 0.061991280901141546),
 ('manager_skill', 0.05491016304210871),
 ('price_per_bed', 0.046090265352921905),
 ('manager_id', 0.044704281430335405),
 ('created_day', 0.041629917092961724),
 ('street_name', 0.03701837058690119),
 ('building_id', 0.036161580525665905),
 ('created_hour', 0.034775596603079405),
 ('num_photos', 0.034573998941612276),
 ('num_features', 0.03326361414207595),
 ('display_address', 0.03149963460423859),
 ('street_address', 0.017261799763122747),
 ('bedrooms', 0.010079883073356348),
 ('with_no_fee', 0.009903485119572612),
 ('bath_per_bed', 0.007988307335634907),
 ('bathrooms', 0.007610311720384044),
 ('with_furnished', 0.005947131013280246),
 ('with_hardwood_floors', 0.005543935690345992),
 ('with_laundry_in_building', 0.004813144167527657),
 ('created_month'

In [31]:
#ananlysis by the feature importance by gain
gain = model.get_score(importance_type='gain')
gain_list = []

total = sum(gain.values())
for key in gain:
    gain[key] = gain[key]*1.0/total
    gain_list.append((key,gain[key]))
sorted(gain_list,key = lambda x:x[1],reverse = True)

[('manager_skill', 0.049392727319142814),
 ('with_simplex', 0.04136660999328723),
 ('with_lowrise', 0.029874770980645254),
 ('with_no_fee', 0.024893364742531798),
 ('price', 0.01965290936412053),
 ('building_id', 0.01895487074210514),
 ('with_hardwood_floors', 0.018863977736613125),
 ('with_hardwood', 0.01800210529058778),
 ('price_per_room', 0.015516410813803495),
 ('price_per_bed', 0.015484434926590396),
 ('bathrooms', 0.015226203061178615),
 ('with_furnished', 0.015194967200289218),
 ('with_reduced_fee', 0.01405390899890974),
 ('with_on_site_garage', 0.013220669724705125),
 ('with_exclusive', 0.012864391643415943),
 ('with_prewar', 0.012853004260313503),
 ('created_hour', 0.01280027006240633),
 ('with_actual_apt._photos', 0.012404827888959653),
 ('with_laundry_in_building', 0.012261014408437573),
 ('with_high_ceiling', 0.01201320100813396),
 ('bedrooms', 0.011481638275033502),
 ('num_photos', 0.011470492300623407),
 ('latitude', 0.01097000961324589),
 ('with_private_outdoor_space', 

In [11]:
#ananlysis by the feature importance by coverage
cover = model.get_score(importance_type='cover')
total = sum(cover.values())
for key in cover:
    cover[key] = cover[key]*1.0/total
cover

{'bathrooms': 0.15003324661763429,
 'bedrooms': 0.11847222747849985,
 'building_id': 0.05966144646752775,
 'created_day': 0.027908091350767217,
 'created_hour': 0.04913703475375256,
 'created_month': 0.015463921187964249,
 'display_address': 0.051917534421511584,
 'latitude': 0.062329192852910546,
 'listing_id': 0.05823796559748455,
 'longitude': 0.05796867229011468,
 'manager_id': 0.0658834209429622,
 'num_description_words': 0.04385875263322271,
 'num_features': 0.05493240649113651,
 'num_photos': 0.053803480057786596,
 'price': 0.07955324745771991,
 'street_address': 0.050839359399004566}